In [1]:
N = 3

In [2]:
# Parameters
N = 1000


In [3]:
import sys
import os
import pandas as pd
import tiktoken
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

#sys.path.append('/home/vs428/Documents/DischargeMe/hail-dischargeme/scoring/scoring.py')

In [4]:
import re
import openai
from datasets import load_dataset
import pandas as pd

In [5]:
%load_ext dotenv
%dotenv /vast/palmer/home.mccleary/vs428/Documents/DischargeMe/hail-dischargeme/.env

In [6]:
def num_tokens_from_string(string: str, encoding_name: str="cl100k_base") -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [7]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)


In [8]:
from IPython.display import display, HTML

def add_line_breaks(text):
    return text.replace('\n', '<br>')


def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

In [9]:
challenge_data_fp = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/"


In [10]:
test_dataset = load_dataset('json', data_files="/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/data_processing/simple_test.json", split="train")

In [11]:
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
engine = "decile-gpt-35-turbo-16k"

# Integrate Service/Ward info

This section will integrate the service-file to match the structure of the note as much as possible. 

In [12]:
target_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge_target.csv.gz", keep_default_na=False)


In [13]:
# ward transfers
transfers = pd.read_pickle('/gpfs/gibbs/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_transfers.pkl')

# higher-level services (ICU, CARD, etc)
services = pd.read_pickle('/gpfs/gibbs/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_services.pkl')


In [14]:
transfers = transfers[transfers['eventtype'] != "discharge"]

In [15]:
transfers = transfers.sort_values(['hadm_id', "intime"])

In [16]:
discharging_transfer = transfers.groupby("hadm_id").last().reset_index()

In [17]:
discharging_service = services.sort_values(['hadm_id', "transfertime"]).groupby("hadm_id").last().reset_index()

In [18]:
test_dataset_df = test_dataset.to_pandas(); test_dataset_df.shape

(14702, 4)

In [19]:
test_dataset_df = test_dataset_df.merge(discharging_transfer[['hadm_id', 'careunit', 'eventtype']], on="hadm_id", how="left"); test_dataset_df.shape

(14702, 6)

In [20]:
test_dataset_df = test_dataset_df.merge(discharging_service[['hadm_id', 'curr_service']], on="hadm_id", how="left"); test_dataset_df

,note_id,hadm_id,input,output,careunit,eventtype,curr_service
0,19766998-DS-20,26231944,\nName: ___ Unit No: ___\n ...,"___ with PMH HCV, ETOH cirrhosis with ascites,...",Med/Surg,transfer,MED
1,10336082-DS-11,28542384,\nName: ___ Unit No: ___\n \nA...,___ yo F with recent lumbar laminectomy c/b MS...,Neurology,admit,MED
2,10481170-DS-22,26489329,\nName: ___ Unit No: ___\n ...,The patient was seen ___ the emergency departm...,Med/Surg/GYN,transfer,SURG
3,11576109-DS-14,22641254,\nName: ___ Unit No: ___\n ...,"Ms. ___ is a ___ woman with a history of T2DM,...",Medicine/Cardiology,transfer,CMED
4,19249697-DS-17,29265750,\nName: ___ Unit No: ...,___ with IDDM and h/o provoked PE on apixiban ...,Vascular,admit,MED
...,...,...,...,...,...,...,...
14697,18995174-DS-17,27445071,\nName: ___ Unit No: ___\...,___ M PMHx dilated non-ischemic cardiomyopathy...,Medicine/Cardiology,admit,CMED
14698,13588195-DS-9,26192891,\nName: ___ Unit No: ___\n ...,"Mr. ___ is a ___ year-old man with CAD, HTN, H...",Med/Surg,admit,MED
14699,10873131-DS-17,26584893,\nName: ___ Unit No: _...,___ year old female with past medical history ...,Medicine,transfer,MED
14700,12332377-DS-11,25623241,\nName: ___ Unit No: ___\n ...,Mr. ___ is a ___ male with a past medical hist...,Med/Surg/Trauma,admit,MED


# N-shot

In [21]:
ex1 = """Mr. ___ presented to the ED s/p mechanical fall with head 
strike. CTH at OSH with subacute right SDH for which patient was 
transferred to ___ for evaluation and neurosurgery was 
consulted on arrival to ___ ED. 

#Right SDH
Patient remained neurologically at his baseline in the ED. He 
was admitted to the neurosurgery service and transferred to the 
floor for continued neurological monitoring. Patient's INR on 
admission was 1.5 and he was given Vitamin K 10mg IV x1 and then 
vitamin K 5mg x2 for a total of 3 doses. Patient's INR was 
monitored closely during hospitalization and down trended 
appropriately. Repeat CTH showed a stable SDH size. Patient 
remained stable neurologically. He remained on TF as he was at 
home and will follow up with SLP and laryngologist outpatient. 
While hospitalized patient remained at his neurological 
baseline. 

#Hx colloid cyst rupture s/p bilateral VPS placement
On patient's initial and repeat NCHCTs, his ventricles were 
shown to be slit. His bilateral VPS were turned up from 1.0 to 
1.5. No scan was repeated following adjustment of his VPS 
settings. He remained neurologically stable following this 
change. 

#Hx of DVT
Patient with a history of bilateral femoral vein occlusive DVTs 
for which he was started on Eliquis. Due to this subacute SDH 
Eliquis was held and bilateral LENIs were obtained to assess 
DVTs. These were negative for thrombus and the Eliquis was 
stopped indefinitely. Patient was placed on ___ 24hrs s/p 
trauma. 

#Disposition
Patient was evaluated by physical therapy who recommended 
___ rehab. Patient was discharged to rehab on ___. 
Patient discharged with follow up instructions with Dr. ___ in 
4 weeks with a repeat non contrast CTH"""

ex2 = """Mr. ___ is a ___ y/o man w/ HTN who presented w/ sudden-onset LUE 
weakness ___ AM). Next day, presented to ED. Pt noted to have 
L facial droop, LUE weakness (more prominent distally) and 
decreased LUE sensation. CTA showed significant R ICA stenosis; 
U/S indicated occlusion between 80 and 100%. MRI brain showed 
acute small infarcts in R frontal and parietal lobes in the MCA 
territory, consistent w/ watershed infarcts. Started on ASA and 
clopidogrel."""

examples = [ex1, ex2]

In [22]:
# take subset
test_dataset_df = test_dataset_df[:N]

In [23]:
gpt_inputs = []
for _, row in test_dataset_df.iterrows():
    message_text = [{"role":"system","content":"You are a physician generating a summary brief hospital course from the patient encounter information given"}]
    
    gpt_service_prompt = {"role":"user",
                     "content":f"""Summarize the following patient hospital encounter into a brief hospital course. All brief hospital courses start with 1-2 sentences of introduction, describing why the patient arrived to the ED and any relevant features of the initial presentation. If the discharging service is primarily medical, break down the rest of the brief hospital course by condition. Include section headers for 'ACTIVE ISSUES:', 'CHRONIC ISSUES:', and 'TRANSITIONAL ISSUES:' where appropriate. In this case, precede each condition with a #. For example, after the introduction, the first section might be '# Right SDH'. 
                     
If the discharging service is primarily surgical, instead break down the subparagraphs by CV, Pulmonary, GI/GU/FEN, ID (stands for infectious disease), HEME (stands for hematology), and Prophylaxis. For example, after the introduction, the first section might be 'HEME: '. Describe the course of events that the patient went through during their stay in sequential order. The goal is to describe the clinical reasoning for various procedures, medications, imaging, and labs as you would to another physician on your care team. Organize your thoughts and then write the brief hospital course.\n\nUse these two brief hospital courses as examples:\n\nEXAMPLE\n-------------\n{'\n\nEXAMPLE\n-------------\n'.join(examples)}\n\n\n\nPATIENT ENCOUNTER INFORMATION:\n----------------\n\nDischarging Service:{row['careunit']}\n\n{row['input']}."""}
    
    message_text.append(gpt_service_prompt)
    
    gpt_inputs.append(message_text)

In [24]:
print(gpt_inputs[1][1]['content'])

Summarize the following patient hospital encounter into a brief hospital course. All brief hospital courses start with 1-2 sentences of introduction, describing why the patient arrived to the ED and any relevant features of the initial presentation. If the discharging service is primarily medical, break down the rest of the brief hospital course by condition. Include section headers for 'ACTIVE ISSUES:', 'CHRONIC ISSUES:', and 'TRANSITIONAL ISSUES:' where appropriate. In this case, precede each condition with a #. For example, after the introduction, the first section might be '# Right SDH'. 
                     
If the discharging service is primarily surgical, instead break down the subparagraphs by CV, Pulmonary, GI/GU/FEN, ID (stands for infectious disease), HEME (stands for hematology), and Prophylaxis. For example, after the introduction, the first section might be 'HEME: '. Describe the course of events that the patient went through during their stay in sequential order. The go

In [25]:
completions = []
for idx, gpt_input in enumerate(gpt_inputs):
    completion = completion_with_backoff(
      engine=engine,
      messages = gpt_input,        
    )
    print(idx)
    completions.append(completion['choices'][0]['message']['content'])

0


1


2


3


4


...

995


996


997


998


999


In [ ]:
completions[187]

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', "rouge2", 'rougeL'], use_stemmer=True)


In [ ]:
all_scores = []
for gen, ref in zip(completions, test_dataset['output']):
    scores = scorer.score(gen, ref)
    print(scores)
    scores_dict = {"rouge1_precision":scores['rouge1'][0], 
                  "rouge1_recall":scores['rouge1'][1], 
                  "rouge1_f1":scores['rouge1'][2], 
                  "rouge2_precision":scores['rouge2'][0], 
                  "rouge2_recall":scores['rouge2'][1], 
                  "rouge2_f1":scores['rouge2'][2], 
                  "rougeL_precision":scores['rougeL'][0], 
                  "rougeL_recall":scores['rougeL'][1], 
                  "rougeL_f1":scores['rougeL'][2], }
    all_scores.append(scores_dict)

In [ ]:
comparison_df = pd.DataFrame([completions, test_dataset['output']]).T.rename({0:"GPT", 1:"gold-standard"}, axis=1)

In [ ]:
test_dataset_df['gpt_completion'] = completions

In [ ]:
test_dataset_df.to_csv("comparison_simple_gpt3.5_2shot_v2.csv")

In [ ]:
comparison_df['GPT'] = comparison_df['GPT'].str.replace("\n", "<br>")
comparison_df['gold-standard'] = comparison_df['gold-standard'].str.replace("\n", "<br>")

In [ ]:
comparison_df.to_html(f"comparison_simple_gpt3.5_2shot_v2.html", escape=False)

In [ ]:
pd.DataFrame.from_records(all_scores).to_csv("scoresv2.csv")

In [ ]:
pd.DataFrame.from_records(all_scores).mean()